<a href="https://colab.research.google.com/github/paulchinnam/GPaulT/blob/main/GPaulT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Avengers Infinity War Script for training

!wget https://raw.githubusercontent.com/paulchinnam/GPaulT/main/input.txt

--2024-03-19 15:39:50--  https://raw.githubusercontent.com/paulchinnam/GPaulT/main/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95297 (93K) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]  93.06K  --.-KB/s    in 0.02s   

2024-03-19 15:39:51 (5.57 MB/s) - ‘input.txt’ saved [95297/95297]



In [ ]:
# Read script in

with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
# Print length of dataset

print("Length of dataset in characters: ", len(text))

Length of dataset in characters:  92772


In [ ]:
# Print first 1000 characters

print(text[:1000])

[radio transmission sound]

[Asgardian PA]: This is the Asgardian refugee vessel Statesman! We are under assault, I repeat, we are under assault – The engines are dead, life support failing! Requesting aid from any vessel within range... We are 22 jump points out of Asgard. Our crew is made up of Asgardian families, we have very few soldiers here! This is not a war craft, I repeat, this is not a war craft!

[Inside the ship, Ebony Maw walks among the bodies of dead Asgardians. He steps over them as he speaks with no mind, as if they were scattered pieces of dirty clothing on a bedroom floor.]

Ebony Maw: Hear me, and rejoice. You have had the privilege of being saved by the Great Titan. You may think this is suffering. No... It is salvation. Universal scales tip toward balance because of your sacrifice. Smile... For even in death, you have become Children of Thanos.

[Loki is seen with the Black Order. He watches Thanos.]

Thanos: [Looking out the large window.] I know what it's like t

In [ ]:
# Get all unique characters in the dataset

chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !%&'(),-.012345678:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz{}–‘“”
84


In [ ]:
# Create a mapping from characters to integers

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[59, 60, 60, 1, 71, 59, 56, 69, 56]
hii there


In [ ]:
# Encode the entire dataset and store in a torch.Tensor

import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # This is what the previous 1000 characters will look like to the GPT

torch.Size([92772]) torch.int64
tensor([49, 69, 52, 55, 60, 66,  1, 71, 69, 52, 65, 70, 64, 60, 70, 70, 60, 66,
        65,  1, 70, 66, 72, 65, 55, 50,  0,  0, 49, 23, 70, 58, 52, 69, 55, 60,
        52, 65,  1, 38, 23, 50, 20,  1, 42, 59, 60, 70,  1, 60, 70,  1, 71, 59,
        56,  1, 23, 70, 58, 52, 69, 55, 60, 52, 65,  1, 69, 56, 57, 72, 58, 56,
        56,  1, 73, 56, 70, 70, 56, 63,  1, 41, 71, 52, 71, 56, 70, 64, 52, 65,
         2,  1, 45, 56,  1, 52, 69, 56,  1, 72, 65, 55, 56, 69,  1, 52, 70, 70,
        52, 72, 63, 71,  8,  1, 31,  1, 69, 56, 67, 56, 52, 71,  8,  1, 74, 56,
         1, 52, 69, 56,  1, 72, 65, 55, 56, 69,  1, 52, 70, 70, 52, 72, 63, 71,
         1, 80,  1, 42, 59, 56,  1, 56, 65, 58, 60, 65, 56, 70,  1, 52, 69, 56,
         1, 55, 56, 52, 55,  8,  1, 63, 60, 57, 56,  1, 70, 72, 67, 67, 66, 69,
        71,  1, 57, 52, 60, 63, 60, 65, 58,  2,  1, 40, 56, 68, 72, 56, 70, 71,
        60, 65, 58,  1, 52, 60, 55,  1, 57, 69, 66, 64,  1, 52, 65, 76,  1, 73,
        

In [ ]:
# Split the data into training and validation sets (90/10 ratio)

n = int(0.9*len(data)) # Calculate 90% of the length of the data
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size + 1]

tensor([49, 69, 52, 55, 60, 66,  1, 71, 69])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
  context = x[:t + 1]
  target = y[t]
  print(f"When input is {context} the target: {target}")

When input is tensor([49]) the target: 69
When input is tensor([49, 69]) the target: 52
When input is tensor([49, 69, 52]) the target: 55
When input is tensor([49, 69, 52, 55]) the target: 60
When input is tensor([49, 69, 52, 55, 60]) the target: 66
When input is tensor([49, 69, 52, 55, 60, 66]) the target: 1
When input is tensor([49, 69, 52, 55, 60, 66,  1]) the target: 71
When input is tensor([49, 69, 52, 55, 60, 66,  1, 71]) the target: 69


In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  # Generate a small batch of data of inputs x and targets y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i + block_size] for i in ix])
  y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])

  return x, y

xb, yb = get_batch('train')
print('Inputs:')
print(xb.shape)
print(xb)
print('Targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t + 1]
    target = yb[b,t]

    print(f"When input is {context.tolist()} the target: {target}")

Inputs:
torch.Size([4, 8])
tensor([[ 0, 42, 59, 52, 65, 66, 70, 20],
        [ 1, 52,  1, 63, 66, 71, 10,  1],
        [ 1, 66, 72, 71, 10,  1, 41, 71],
        [58,  1, 52,  1, 65, 56, 74,  1]])
Targets:
torch.Size([4, 8])
tensor([[42, 59, 52, 65, 66, 70, 20,  1],
        [52,  1, 63, 66, 71, 10,  1,  6],
        [66, 72, 71, 10,  1, 41, 71, 69],
        [ 1, 52,  1, 65, 56, 74,  1, 59]])
----
When input is [0] the target: 42
When input is [0, 42] the target: 59
When input is [0, 42, 59] the target: 52
When input is [0, 42, 59, 52] the target: 65
When input is [0, 42, 59, 52, 65] the target: 66
When input is [0, 42, 59, 52, 65, 66] the target: 70
When input is [0, 42, 59, 52, 65, 66, 70] the target: 20
When input is [0, 42, 59, 52, 65, 66, 70, 20] the target: 1
When input is [1] the target: 52
When input is [1, 52] the target: 1
When input is [1, 52, 1] the target: 63
When input is [1, 52, 1, 63] the target: 66
When input is [1, 52, 1, 63, 66] the target: 71
When input is [1, 52, 1, 6